### 練習問題2.42
"8クイーンパズル"とは、どのクイーンもほかのクイーンの利き筋に⼊らない  
(つまり、同じ⾏・列・対⾓線の上に⼆つのクイーンがあるということがないようにする)ように  
8個のクイーンをチェス盤の上に置く⽅法を問うものである。  
図2.8に考えられる解のひとつを⽰す。  
このパズルを解くひとつのやり⽅として、  
各列にひとつのクイーンを置きながらチェス盤を横に移動していくというものがある。  
$k−1$個のクイーンを置いた状態では、$k$個⽬のクイーンは、すでに盤上にあるどのクイーンも利き筋に⼊らない位置に置く必要がある。  
この解き⽅は再帰的に定式化できる。最初の$k−1$列に$k−1$個のクイーンを置くパターンをすべて⽣成済みだと想定する。  
それぞれのパターンに対して、$k$列⽬のそれぞれの⾏にクイーンを置いていって、  
位置の集合を拡張したものを⽣成する。  
次にこれらをフィルタして、$k$列⽬のクイーンがほかのクイ ーンに利かないものだけを残す。  
こうすると、最初の$k$列に$k$個のクイーンを置くすべてのパターンの列ができる。  
このプロセスを続けると、パズルの解答のひとつだけではなく、すべての解答が得られる。  
この解き⽅をqueensという⼿続きとして実装し、  
$n \times n$のチェス盤に$n$個のクイーンを置くという問題に対するすべての答えの列を返すようにする。  
queensは、盤の最初の$k$列にクイーンを置くすべてのパターンの列を返す内部⼿続きqueen-colsを持っている。

    (define (queens board-size)
      (define (queen-cols k)
        (if (= k 0) (list empty-board)
            (filter
             (lambda (positions) (safe? k positions))
              (flatmap (lambda (rest-of-queens)
                        (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
                             (enumerate-interval 1 board-size))
                        )
                      (queen-cols (- k 1))
              )
             )
            )
        )
      (queen-cols board-size)
      )
      
この⼿続きの中で、rest-of-queensは最初の$k−1$列に$k−1$個のクイーンを置くパターンのひとつで、  
new-rowは$k$列⽬のクイーンを置く候補となる⾏である。  
盤上の位置集合に対する表現⽅法と、位置集合に新しい⾏・列の位置を追加するadjoin-position⼿続きと  
位置の空集合を表すempty-boardを実装し、プログラムを完成させよ。  
また、位置集合に対して、$k$列⽬のクイーンがほかのクイーンに利いていないかを調べるsafe?⼿続きも書く必要がある  
(新しいクイーンの利きだけをチェックすればいいということに注意。  
ほかのクイーンは互いに利いていないことが保証済みである)。 

In [1]:
; 問題文のqueens手続き
; 以下の定義が必要
; ・empty-board
; ・safe?手続き
; ・adjoin-position手続き
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          (flatmap (lambda (rest-of-queens)
                    (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
                         (enumerate-interval 1 board-size))
                    )
                  (queen-cols (- k 1))
          )
         )
        )
    )
  (queen-cols board-size)
  )

(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

; フィルタリング
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        ((predicate (car sequence))(cons (car sequence) (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence))))
  )

; 回答1
(define empty-board ())


In [2]:
(define (make-pos x y)(cons x y))
(define (x-pos p)(car p))
(define (y-pos p)(cdr p))

; 回答2
; adjoin-position手続きの呼び出し方が決まっているため、
; この回答しかないはず。
(define (adjoin-position new-row k rest-of-queens)
  (append rest-of-queens (list (make-pos k new-row)))
  )
  
(define k 4)
(define board-size 4)
(define rest-of-queens (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1)))
(map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
     (enumerate-interval 1 board-size))

;(x-pos (make-pos 1 2))
;(y-pos (make-pos 1 2))


(((1 . 2) (2 . 4) (3 . 1) (4 . 1)) ((1 . 2) (2 . 4) (3 . 1) (4 . 2)) ((1 . 2) (2 . 4) (3 . 1) (4 . 3)) ((1 . 2) (2 . 4) (3 . 1) (4 . 4)))

In [107]:

(define (same-row? row l)
  (cond ((null? l) #f)
        (else (or
                 (= row (y-pos (car l)))
                 (same-row? row (cdr l))
               )
          )
      )
  )

; 各行異なることは確認できる。
; 各列について位置情報をリストに追加するので、
; 列については元々異なっているので、チェックしていない。
; 対角線のチェックが出来ていない。
(define (safe? k positions)
  (if (<= k 1) #t
      (if (null? positions) #t
          (if (same-row? (y-pos (car positions)) (cdr positions)) #f
              (safe? k (cdr positions))
           )
      )
    )
  )

; k列目の位置のy座標を返す
(define (get-k-y k positions)
  (define (iter count pos)
    (if (>= count k) (y-pos (car pos))
        (iter (+ count 1) (cdr pos))
        )
    )
  (iter 1 positions)
  )

(define (on-diagonal? k positions)
  (let ((k-y-pos (get-k-y k positions)))
    (define (iter count pos)
      (if (>= count k) #f
        (if (null? pos) #f
            (let ((y (y-pos (car pos))))
              (let ((y-upper (- y (- k count)))
                    (y-lower (+ y (- k count))))
                (if (or (= k-y-pos y) (= k-y-pos y-upper) (= k-y-pos y-lower)) #t
                    (iter (+ count 1) (cdr pos))
                    )
                )
              )
            )
        )
      )
      (iter 1 positions)
    )
  )

(define (safe? k positions)
  (let ((k-y-pos (get-k-y k positions)))
    (define (iter count pos)
      (if (>= count k) #t
        (if (null? pos) #t
            (let ((y (y-pos (car pos))))
              (let ((y-upper (- y (- k count)))
                    (y-lower (+ y (- k count))))
                (if (or (= k-y-pos y) (= k-y-pos y-upper) (= k-y-pos y-lower)) #f
                    (iter (+ count 1) (cdr pos))
                    )
                )
              )
            )
        )
      )
      (iter 1 positions)
    )
  )


;(define (safe? k positions) #t)
;(car (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1)))
;(cdr (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1)))
;(same-col? 3 (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1)))

;(define (diagonal k size positions)
;  
;  )

;(get-k-y 4 (list (make-pos 1 1) (make-pos 2 2) (make-pos 3 3) (make-pos 4 4)))
(safe? 4 (list (make-pos 1 1) (make-pos 2 2) (make-pos 3 3) (make-pos 4 4)))
;(safe? 3 (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1) (make-pos 4 3)))
;(safe? 4 (list (make-pos 1 2) (make-pos 2 4) (make-pos 3 1) (make-pos 4 3)))

#f

In [108]:
(queens 3)

()

In [109]:
(queens 4)

(((1 . 2) (2 . 4) (3 . 1) (4 . 3)) ((1 . 3) (2 . 1) (3 . 4) (4 . 2)))

In [110]:
(queens 5)

(((1 . 1) (2 . 3) (3 . 5) (4 . 2) (5 . 4)) ((1 . 1) (2 . 4) (3 . 2) (4 . 5) (5 . 3)) ((1 . 2) (2 . 4) (3 . 1) (4 . 3) (5 . 5)) ((1 . 2) (2 . 5) (3 . 3) (4 . 1) (5 . 4)) ((1 . 3) (2 . 1) (3 . 4) (4 . 2) (5 . 5)) ((1 . 3) (2 . 5) (3 . 2) (4 . 4) (5 . 1)) ((1 . 4) (2 . 1) (3 . 3) (4 . 5) (5 . 2)) ((1 . 4) (2 . 2) (3 . 5) (4 . 3) (5 . 1)) ((1 . 5) (2 . 2) (3 . 4) (4 . 1) (5 . 3)) ((1 . 5) (2 . 3) (3 . 1) (4 . 4) (5 . 2)))

In [111]:
(queens 6)

(((1 . 2) (2 . 4) (3 . 6) (4 . 1) (5 . 3) (6 . 5)) ((1 . 3) (2 . 6) (3 . 2) (4 . 5) (5 . 1) (6 . 4)) ((1 . 4) (2 . 1) (3 . 5) (4 . 2) (5 . 6) (6 . 3)) ((1 . 5) (2 . 3) (3 . 1) (4 . 6) (5 . 4) (6 . 2)))

In [112]:
(queens 7)

(((1 . 1) (2 . 3) (3 . 5) (4 . 7) (5 . 2) (6 . 4) (7 . 6)) ((1 . 1) (2 . 4) (3 . 7) (4 . 3) (5 . 6) (6 . 2) (7 . 5)) ((1 . 1) (2 . 5) (3 . 2) (4 . 6) (5 . 3) (6 . 7) (7 . 4)) ((1 . 1) (2 . 6) (3 . 4) (4 . 2) (5 . 7) (6 . 5) (7 . 3)) ((1 . 2) (2 . 4) (3 . 1) (4 . 7) (5 . 5) (6 . 3) (7 . 6)) ((1 . 2) (2 . 4) (3 . 6) (4 . 1) (5 . 3) (6 . 5) (7 . 7)) ((1 . 2) (2 . 5) (3 . 1) (4 . 4) (5 . 7) (6 . 3) (7 . 6)) ((1 . 2) (2 . 5) (3 . 3) (4 . 1) (5 . 7) (6 . 4) (7 . 6)) ((1 . 2) (2 . 5) (3 . 7) (4 . 4) (5 . 1) (6 . 3) (7 . 6)) ((1 . 2) (2 . 6) (3 . 3) (4 . 7) (5 . 4) (6 . 1) (7 . 5)) ((1 . 2) (2 . 7) (3 . 5) (4 . 3) (5 . 1) (6 . 6) (7 . 4)) ((1 . 3) (2 . 1) (3 . 6) (4 . 2) (5 . 5) (6 . 7) (7 . 4)) ((1 . 3) (2 . 1) (3 . 6) (4 . 4) (5 . 2) (6 . 7) (7 . 5)) ((1 . 3) (2 . 5) (3 . 7) (4 . 2) (5 . 4) (6 . 6) (7 . 1)) ((1 . 3) (2 . 6) (3 . 2) (4 . 5) (5 . 1) (6 . 4) (7 . 7)) ((1 . 3) (2 . 7) (3 . 2) (4 . 4) (5 . 6) (6 . 1) (7 . 5)) ((1 . 3) (2 . 7) (3 . 4) (4 . 1) (5 . 5) (6 . 2) (7 . 6)) ((1 . 4) (2 .

In [113]:
(queens 8)

(((1 . 1) (2 . 5) (3 . 8) (4 . 6) (5 . 3) (6 . 7) (7 . 2) (8 . 4)) ((1 . 1) (2 . 6) (3 . 8) (4 . 3) (5 . 7) (6 . 4) (7 . 2) (8 . 5)) ((1 . 1) (2 . 7) (3 . 4) (4 . 6) (5 . 8) (6 . 2) (7 . 5) (8 . 3)) ((1 . 1) (2 . 7) (3 . 5) (4 . 8) (5 . 2) (6 . 4) (7 . 6) (8 . 3)) ((1 . 2) (2 . 4) (3 . 6) (4 . 8) (5 . 3) (6 . 1) (7 . 7) (8 . 5)) ((1 . 2) (2 . 5) (3 . 7) (4 . 1) (5 . 3) (6 . 8) (7 . 6) (8 . 4)) ((1 . 2) (2 . 5) (3 . 7) (4 . 4) (5 . 1) (6 . 8) (7 . 6) (8 . 3)) ((1 . 2) (2 . 6) (3 . 1) (4 . 7) (5 . 4) (6 . 8) (7 . 3) (8 . 5)) ((1 . 2) (2 . 6) (3 . 8) (4 . 3) (5 . 1) (6 . 4) (7 . 7) (8 . 5)) ((1 . 2) (2 . 7) (3 . 3) (4 . 6) (5 . 8) (6 . 5) (7 . 1) (8 . 4)) ((1 . 2) (2 . 7) (3 . 5) (4 . 8) (5 . 1) (6 . 4) (7 . 6) (8 . 3)) ((1 . 2) (2 . 8) (3 . 6) (4 . 1) (5 . 3) (6 . 5) (7 . 7) (8 . 4)) ((1 . 3) (2 . 1) (3 . 7) (4 . 5) (5 . 8) (6 . 2) (7 . 4) (8 . 6)) ((1 . 3) (2 . 5) (3 . 2) (4 . 8) (5 . 1) (6 . 7) (7 . 4) (8 . 6)) ((1 . 3) (2 . 5) (3 . 2) (4 . 8) (5 . 6) (6 . 4) (7 . 7) (8 . 1)) ((1 . 3) 